In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from scipy.spatial.distance import cdist #for euclidean distance of consistency t
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import pickle
from collections import defaultdict
import random
import statistics as st #for the mode
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import kmeans_clustering, sampen, chi2_distance, ts_visual, correct_key, chi2_distance, most_frequent
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


In [3]:
#fix random seed for reproducibility
rs = 0

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Define parameters

In [4]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
li_binmn = config.li_binmn
pal_class_treat = config.pal_class_treat
pal_treat = config.pal_treat
pal_class = config.pal_class
birth_date = config.birth_date
dico_night_hour = config.dico_night_hour
pal_interintre_treatment = config.pal_interintre_treatment
max_date_adaptability = config.max_date_adaptability
dico_pen_tr = config.dico_pen_tr
path_extracted_data_visual = os.path.join(path_extracted_data,'visual')
path_extracted_data_visual_corr = os.path.join(path_extracted_data,'visual','correlation')
path_extracted_data_visual_adap = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability')
path_extracted_data_visual_adap_clustering = os.path.join(path_extracted_data_visual_adap,'clustering')
for nbr_bin in li_binmn:
    path_extracted_data_visual_adap_binmn = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability',str(nbr_bin))
    #create a director if not existing
    if not os.path.exists(path_extracted_data_visual_adap_binmn):
        os.makedirs(path_extracted_data_visual_adap_binmn)

change the configuration file if not done yet!


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
li_nbr_cluster = range(2,100)
V = 'chi2distance_ALL_15'
name_clustering = 'ALL_MLPS_99'
path_extracted_data_visual_adap_clustering_V = os.path.join(path_extracted_data_visual_adap_clustering, V+'_ALL_MLPS')

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [6]:
li_selected_hens = pickle.load(open(os.path.join(path_extracted_data_visual_adap, 'li_selected_hens.pkl'), 'rb'))
print(len(li_selected_hens))
#for now we have only distance restricted to 70 animals
#compute the CHI2DISTANCE & DTW for each bird with all other birds (because we need this exact same pair to do the visual)
#rv = 0
#random.seed(rv)
#nbr_hens = 70
#randomly pick more than one element from the list without repeating elements.
#li_selected_hens = random.sample(li_selected_hens, nbr_hens)

131


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#HA&KBF&weight
df_HA = pd.read_csv(os.path.join(path_extracted_data, 'df_all_HA.csv'), sep=';', parse_dates=['date'], dayfirst=True) 
print(df_HA.shape)
df_HA = df_HA[df_HA['HenID'].isin(li_selected_hens)]
print(df_HA.shape)
df_HA.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(689, 45)
(458, 45)


,Person_toe,Time,PenID,FocalLegringName,Bird type,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,initial_path_img,actual_path_img,imageID,random_order1,severity,has_gap,BirdID,uncertainty (0:certain:1 random),Info,backpack
0,C,NaN,pen10,41black,NaN,15.0,NaN,50.0,0.0,0.0,...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,169.0,417.0,64.0,0.0,41black10bs,NaN,"['', '41', 'black', '10', 'bs']",bs
1,M,10:43,pen4,17orange,normal,25.0,NaN,0.0,0.0,0.0,...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,205.0,518.0,0.0,0.0,17orange4ws,NaN,"['', '17', 'orange', '4', 'ws']",ws
2,C,09:34,pen9,53grey,NaN,10.0,NaN,0.0,0.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#dailyMVT var
df_MVT = pd.read_csv(os.path.join(path_extracted_data, 'daily_ALL_Variable_Tranformed_SNA_clID.csv'), sep=';', 
                     parse_dates=['level'], dayfirst=True) 
print(df_MVT.shape)
df_MVT = df_MVT[df_MVT['HenID'].isin(li_selected_hens)]
print(df_MVT.shape)
df_MVT.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(42300, 223)
(27995, 223)


C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (123,135,172,173,174,175,176,177,179,180,181,182,183,184,185,186,187,188) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,Total_number_zone,Max_duration_zones,...,betweeness_DTW_30_all,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all,ALL_MLPS_99,ALL_MLPS_10,ALL_MLPS_40
0,hen_1,2020-09-30,0.0,18886.0,3488.0,515.0,5911.0,28800.0,4.0,2_Zone,...,0.002710,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500,NaN,NaN,NaN
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,3.0,5_Zone,...,0.183609,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952,NaN,NaN,NaN
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,2.0,5_Zone,...,0.000955,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401,NaN,NaN,NaN


# Processing

In [10]:
#compeck: HA1: as worse time for this
#overall pododeratitis: should be worst over time, so only HA4
df_HA['Injuries'] = df_HA.apply(lambda x: x['LFinjuries']+x['RFinjuries'], axis=1)
df_HA['Pododermatitis'] = df_HA.apply(lambda x: x['RFPododermatitis']+x['LFPododermatitis'], axis=1)

#overall feather as in eneral have the same tendency: only from HA4: as should be worse over time
df_HA['Feathers'] = df_HA.apply(lambda x: np.mean([x['NeckFeather'],x['BreastFeather'],x['CloacaFeather'],
                                                         x['BackFeather'],x['WingFeather'],x['TailFeather']]), axis=1)
display(df_HA[df_HA['Feathers'].isnull()].shape)
display(df_HA[~df_HA['Feathers'].isnull()].shape)
df_HA.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(132, 48)

(326, 48)

,Person_toe,Time,PenID,FocalLegringName,Bird type,CombPecks,Toes,RFPododermatitis,RFBumblefoot,RFinjuries,...,random_order1,severity,has_gap,BirdID,uncertainty (0:certain:1 random),Info,backpack,Injuries,Pododermatitis,Feathers
0,C,NaN,pen10,41black,NaN,15.0,NaN,50.0,0.0,0.0,...,417.0,64.0,0.0,41black10bs,NaN,"['', '41', 'black', '10', 'bs']",bs,0.0,75.0,84.333333
1,M,10:43,pen4,17orange,normal,25.0,NaN,0.0,0.0,0.0,...,518.0,0.0,0.0,17orange4ws,NaN,"['', '17', 'orange', '4', 'ws']",ws,0.0,0.0,NaN
2,C,09:34,pen9,53grey,NaN,10.0,NaN,0.0,0.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.0,100.000000


In [11]:
#add np.nan for the WG var when the WG was close
df_MVT['in_WG_15mnAfterOpening_whenopen'] = df_MVT.apply(lambda x: x['in_WG_15mnAfterOpening'] if x['level']>config.date_first_opening_WG else np.nan, axis=1)
display(df_MVT['in_WG_15mnAfterOpening'].value_counts())
display(df_MVT['in_WG_15mnAfterOpening_whenopen'].value_counts())
df_MVT.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.0    18163
1.0     3703
Name: in_WG_15mnAfterOpening, dtype: int64

0.0    17310
1.0     3703
Name: in_WG_15mnAfterOpening_whenopen, dtype: int64

,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,Total_number_zone,Max_duration_zones,...,degree_DTW_30_1_Zone,degree_DTW_30_2_Zone,degree_DTW_30_3_Zone,degree_DTW_30_4_Zone,degree_DTW_30_5_Zone,degree_DTW_30_all,ALL_MLPS_99,ALL_MLPS_10,ALL_MLPS_40,in_WG_15mnAfterOpening_whenopen
0,hen_1,2020-09-30,0.0,18886.0,3488.0,515.0,5911.0,28800.0,4.0,2_Zone,...,1.000000,0.131944,0.208333,0.826389,0.437500,0.312500,NaN,NaN,NaN,NaN
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,3.0,5_Zone,...,0.993197,0.047619,0.176871,0.802721,0.115646,0.380952,NaN,NaN,NaN,NaN
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,2.0,5_Zone,...,1.000000,0.564626,0.646259,0.802721,0.455782,0.503401,NaN,NaN,NaN,NaN


In [12]:
#merge the least frequent clusterID together for now
nbr_cluster2keep = 10
li_clmostimportant = df_MVT['ALL_MLPS_99'].value_counts().reset_index()['index'][0:nbr_cluster2keep].tolist()
df_MVT['ALL_MLPS_99_'+str(nbr_cluster2keep)+'cat'] = df_MVT['ALL_MLPS_99'].map(lambda x: 'clID'+str(int(x)) if x in li_clmostimportant else 'clIDother')
df_MVT['ALL_MLPS_99_'+str(nbr_cluster2keep)+'cat'].value_counts()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


clIDother    23664
clID67        1904
clID22         727
clID3          475
clID15         350
clID0          274
clID6          157
clID7          133
clID21         131
clID52         112
clID33          68
Name: ALL_MLPS_99_10cat, dtype: int64

# Select var and create DF for models

In [188]:
li_HA4 = ['Feathers','Pododermatitis','severity','Injuries']
li_HA1 = ['CombPecks','Injuries']
li_mvtvar = ['perc_duration_5_Zone','perc_duration_4_Zone','activity_50percentile_sec','kurtosis_duration','median_duration',
             'in_WG_15mnAfterOpening_whenopen', 'vertical_travel_distance','empproba_3_Zone',
             'empproba_4_Zone','is_mvt_night','closeness_DTW_30_2_Zone','closeness_DTW_30_5_Zone','degree_DTW_30_all',
             'ALL_MLPS_99_'+str(nbr_cluster2keep)+'cat']
print('You have selected %d mvt variables, without counting for several categories in categorical variables (as they would be aggregated anyway)'%len(li_mvtvar))

You have selected 14 mvt variables, without counting for several categories in categorical variables (as they would be aggregated anyway)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
#DF of var mean for model. We will have 140 obs, so lets keep at most 14 var for now (10 obs/var)
df_mean = df_MVT.groupby('HenID').agg(
           perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanmean(x)),
           perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanmean(x)),
           activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
           kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
           median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanmean(x)),
           in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterOpening_whenopen', aggfunc=lambda x: np.nanmean(x)),
           vertical_travel_distance_=pd.NamedAgg(column='vertical_travel_distance', aggfunc=lambda x: np.nanmean(x)),
           empproba_3_Zone_=pd.NamedAgg(column='empproba_3_Zone', aggfunc=lambda x: np.nanmean(x)),
           empproba_4_Zone_=pd.NamedAgg(column='empproba_4_Zone', aggfunc=lambda x: np.nanmean(x)),
           is_mvt_night_=pd.NamedAgg(column='is_mvt_night', aggfunc=lambda x: np.nanmean(x)),
           closeness_DTW_30_2_Zone_=pd.NamedAgg(column='closeness_DTW_30_2_Zone', aggfunc=lambda x: np.nanmean(x)),
           closeness_DTW_30_5_Zone_=pd.NamedAgg(column='closeness_DTW_30_5_Zone', aggfunc=lambda x: np.nanmean(x)),
           degree_DTW_30_all_=pd.NamedAgg(column='degree_DTW_30_all', aggfunc=lambda x: np.nanmean(x)),
           ALL_MLPS_99_10cat_=pd.NamedAgg(column='ALL_MLPS_99_10cat', aggfunc=lambda x: st.mode(x))).reset_index()
print(df_mean['ALL_MLPS_99_10cat_'].value_counts())
print(df_mean.shape)
df_mean.head(3)

clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
(131, 15)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,perc_duration_5_Zone_,perc_duration_4_Zone_,activity_50percentile_sec_,kurtosis_duration_,median_duration_,in_WG_15mnAfterOpening_whenopen_,vertical_travel_distance_,empproba_3_Zone_,empproba_4_Zone_,is_mvt_night_,closeness_DTW_30_2_Zone_,closeness_DTW_30_5_Zone_,degree_DTW_30_all_,ALL_MLPS_99_10cat_
0,hen_1,44.009917,6.431598,41204.538462,7.163875,7506.738532,0.375000,33.155963,30.728716,44.006836,0.014019,0.314578,0.411557,0.110919,clIDother
1,hen_10,21.376441,7.769693,42248.850746,10.793697,1604.170732,0.010101,41.336585,14.610309,45.659653,0.466981,0.261282,0.421689,0.203856,clIDother
2,hen_101,9.878261,4.813588,42046.224490,7.641380,541.061224,0.142857,66.464286,14.303091,44.334272,0.263415,0.298366,0.458778,0.243761,clIDother


In [81]:
df_variability = df_MVT.groupby('HenID').agg(
           perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanvar(x)),
           perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanvar(x)),
           activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanvar(x)),
           kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanvar(x)),
           median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanvar(x)),
           in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterOpening_whenopen', aggfunc=lambda x: np.nanvar(x)),
           vertical_travel_distance_=pd.NamedAgg(column='vertical_travel_distance', aggfunc=lambda x: np.nanvar(x)),
           empproba_3_Zone_=pd.NamedAgg(column='empproba_3_Zone', aggfunc=lambda x: np.nanvar(x)),
           empproba_4_Zone_=pd.NamedAgg(column='empproba_4_Zone', aggfunc=lambda x: np.nanvar(x)),
           is_mvt_night_=pd.NamedAgg(column='is_mvt_night', aggfunc=lambda x: np.nanmean(x)),
           closeness_DTW_30_2_Zone_=pd.NamedAgg(column='closeness_DTW_30_2_Zone', aggfunc=lambda x: np.nanvar(x)),
           closeness_DTW_30_5_Zone_=pd.NamedAgg(column='closeness_DTW_30_5_Zone', aggfunc=lambda x: np.nanvar(x)),
           degree_DTW_30_all_=pd.NamedAgg(column='degree_DTW_30_all', aggfunc=lambda x: np.nanvar(x)),
           ALL_MLPS_99_10cat_=pd.NamedAgg(column='ALL_MLPS_99_10cat', aggfunc=lambda x: len(set(x)))).reset_index()
print(df_variability['ALL_MLPS_99_10cat_'].value_counts())
print(df_variability.shape)
df_variability.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


7     24
8     23
6     22
5     18
9     16
4     16
3      7
2      3
10     2
Name: ALL_MLPS_99_10cat_, dtype: int64
(131, 15)


,HenID,perc_duration_5_Zone_,perc_duration_4_Zone_,activity_50percentile_sec_,kurtosis_duration_,median_duration_,in_WG_15mnAfterOpening_whenopen_,vertical_travel_distance_,empproba_3_Zone_,empproba_4_Zone_,is_mvt_night_,closeness_DTW_30_2_Zone_,closeness_DTW_30_5_Zone_,degree_DTW_30_all_,ALL_MLPS_99_10cat_
0,hen_1,1049.156191,17.488232,1.866727e+07,31.133606,2.150284e+08,0.234375,460.516960,248.644835,292.833700,0.014019,0.003788,0.006001,0.021862,5
1,hen_10,638.665898,15.766480,3.608011e+07,111.017118,2.597313e+07,0.009999,274.447686,371.226230,486.449574,0.466981,0.008499,0.004618,0.023741,5
2,hen_101,126.388817,6.644125,2.500379e+07,45.929870,9.944459e+04,0.122449,387.422194,233.867149,245.903572,0.263415,0.003107,0.002755,0.011140,10


# RF regressor

### choose dataset and var

In [156]:
li_mvtvar_model = [x+'_' for x in li_mvtvar if x!='ALL_MLPS_99_10cat'] #drop ALL_MLPS_99_10cat as no variation
print(df_HA[df_HA['HAID']=='HA4'].shape)
df_model = pd.merge(df_mean, df_HA[df_HA['HAID']=='HA4'], on=['HenID'], how='inner') #df_variability or df_mean
print(df_model.shape)
df_model[['HenID']+li_mvtvar_model].head(2)

(98, 48)
(98, 62)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,perc_duration_5_Zone_,perc_duration_4_Zone_,activity_50percentile_sec_,kurtosis_duration_,median_duration_,in_WG_15mnAfterOpening_whenopen_,vertical_travel_distance_,empproba_3_Zone_,empproba_4_Zone_,is_mvt_night_,closeness_DTW_30_2_Zone_,closeness_DTW_30_5_Zone_,degree_DTW_30_all_
0,hen_10,638.665898,15.766480,3.608011e+07,111.017118,2.597313e+07,0.009999,274.447686,371.226230,486.449574,0.466981,0.008499,0.004618,0.023741
1,hen_101,126.388817,6.644125,2.500379e+07,45.929870,9.944459e+04,0.122449,387.422194,233.867149,245.903572,0.263415,0.003107,0.002755,0.011140


In [157]:
li_mvtvar_model = ['perc_duration_5_Zone_','kurtosis_duration_','vertical_travel_distance_','perc_duration_4_Zone_',
                  'median_duration_','degree_DTW_30_all_']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### model fitting

In [158]:
#########################  Split into training and validation (for parameter tuning) & dummies
#split into train, test and put into good format
X_train = df_model[li_mvtvar_model].values
y_train = df_model['severity'].values
y_train = np.ravel(y_train) #from (x,1) to (x,)
print(X_train.shape, y_train.shape)
val_size = 0.2
X_train_tuning, X_val_tuning, y_train_tuning, y_val_tuning = train_test_split(X_train, y_train, 
                                                                              test_size=val_size, random_state=rs)
#var name
var_name = df_model[li_mvtvar_model].columns

#choose model using 2cross validation on the validating dataset
P = {'max_features': ['sqrt','log2'], #nbr of features to consider when looking for the best split
      'max_depth': [3,4,5,6,7,8,9,10,11],
      'min_samples_leaf': [2,3,4],#min samples at a leaf node
      'min_samples_split':[2,3,4,5], #min nbr of samples required to split a node, we may want this to be bigger for RF (more complex trees)
      'n_estimators':[20,30,40],#nbr of trees in the forest default=100
      'min_impurity_decrease':[x for x in np.linspace(start = 0.00001, stop = 0.001, num = 10)]}
#define the model with all these options and fit it (as many combination as asked for). 
#refit: fit an estimator using the best found parameters on the whole dataset, then we can extract it with best_estimator_
mod = GridSearchCV(estimator = RandomForestRegressor(), param_grid = P, cv = 2, n_jobs = -1, verbose = 2, refit=True) 
#train best model
mod.fit(X_train_tuning, y_train_tuning)
#extract best already fitted model and save it 
mod_final = mod.best_estimator_
#joblib.dump(mod_final, os.path.join(path_extracted_data_cleaning_model, 'selected_randomforest.pkl') )
display(mod_final)

# Evaluate performance
#final score: Return the coefficient of determination of the prediction
#% of the variance of the dependent variable that is explained by the variance of the ind. variable
print('coefficient of determinationon val data: ',round(mod_final.score(X_val_tuning, y_val_tuning),3))
print('coefficient of determination on training data: ',round(mod_final.score(X_train_tuning, y_train_tuning),3))

#Mean absolute error
#Baseline errors, and display average baseline error
baseline_errors = abs(np.array([df_model['severity'].mean() for i in range(0,len(y_val_tuning))])-y_val_tuning)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))
predictions = mod_final.predict(X_val_tuning)
errors = abs(predictions - y_val_tuning)
print('Mean Absolute Error:', round(np.mean(errors), 2))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s


(98, 6) (98,)
Fitting 2 folds for each of 6480 candidates, totalling 12960 fits


[Parallel(n_jobs=-1)]: Done 432 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 1244 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 2376 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 3836 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 5616 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 7724 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 10152 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 12905 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done 12960 out of 12960 | elapsed:   59.3s finished


RandomForestRegressor(max_depth=10, max_features='log2',
                      min_impurity_decrease=0.001, min_samples_leaf=2,
                      min_samples_split=4, n_estimators=20)

coefficient of determinationon val data:  -0.2
coefficient of determination on training data:  0.698
Average baseline error:  13.35
Mean Absolute Error: 14.43


# Model on windowed period

In [37]:
li_date = set(df_MVT['level'].tolist()) #unique lead to np format of date!
li_date = sorted(li_date)
nbr_days = 14
li_date = li_date[nbr_days:]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
li_dico = []
for date_ in tqdm.tqdm(li_date):
    print(date_)
    #DF of var mean for model. We will have 140 obs, so lets keep at most 14 var for now (10 obs/var)
    df_mean = df_MVT[(df_MVT['level']<=date_)&(df_MVT['level']>date_-dt.timedelta(days=nbr_days))].groupby('HenID').agg(
               perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanmean(x)),
               perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanmean(x)),
               activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
               kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
               median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanmean(x)),
               in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterOpening_whenopen', aggfunc=lambda x: np.nanmean(x)),
               vertical_travel_distance_=pd.NamedAgg(column='vertical_travel_distance', aggfunc=lambda x: np.nanmean(x)),
               empproba_3_Zone_=pd.NamedAgg(column='empproba_3_Zone', aggfunc=lambda x: np.nanmean(x)),
               empproba_4_Zone_=pd.NamedAgg(column='empproba_4_Zone', aggfunc=lambda x: np.nanmean(x)),
               is_mvt_night_=pd.NamedAgg(column='is_mvt_night', aggfunc=lambda x: np.nanmean(x)),
               closeness_DTW_30_2_Zone_=pd.NamedAgg(column='closeness_DTW_30_2_Zone', aggfunc=lambda x: np.nanmean(x)),
               closeness_DTW_30_5_Zone_=pd.NamedAgg(column='closeness_DTW_30_5_Zone', aggfunc=lambda x: np.nanmean(x)),
               degree_DTW_30_all_=pd.NamedAgg(column='degree_DTW_30_all', aggfunc=lambda x: np.nanmean(x)),
               ALL_MLPS_99_10cat_=pd.NamedAgg(column='ALL_MLPS_99_10cat', aggfunc=lambda x: st.mode(x))).reset_index()
    print(df_mean['ALL_MLPS_99_10cat_'].value_counts())

    #chose dataset
    df_model = pd.merge(df_mean, df_HA[df_HA['HAID']=='HA4'], on=['HenID'], how='inner') #df_variability or df_mean
    li_mvtvar_model = ['perc_duration_5_Zone_','vertical_travel_distance_','perc_duration_4_Zone_',
                      'degree_DTW_30_all_']
    #drop nan
    df_model = df_model[li_mvtvar_model+['severity']].dropna()

    #########################  Split into training and validation (for parameter tuning) & dummies
    #split into train, test and put into good format
    X_train = df_model[li_mvtvar_model].values
    y_train = df_model['severity'].values
    y_train = np.ravel(y_train) #from (x,1) to (x,)
    val_size = 0.2
    X_train_tuning, X_val_tuning, y_train_tuning, y_val_tuning = train_test_split(X_train, y_train, 
                                                                                  test_size=val_size, random_state=rs)
    #var name
    var_name = df_model[li_mvtvar_model].columns
    #fit model
    #define model and fit on training dataset
    mod_final = RandomForestRegressor(max_depth=4, max_features='log2',
                      min_impurity_decrease=0.001, min_samples_leaf=2,
                      min_samples_split=4, n_estimators=30)
    mod_final.fit(X_train_tuning, y_train_tuning)

    #Mean absolute error
    #Baseline errors, and display average baseline error
    baseline_errors = abs(np.array([df_model['severity'].mean() for i in range(0,len(y_val_tuning))])-y_val_tuning)
    predictions = mod_final.predict(X_val_tuning)
    errors = abs(predictions - y_val_tuning)
    li_dico.append({'MAE':round(np.mean(errors), 2), 'Average baseline error':round(np.mean(baseline_errors), 2), 
                    'R2_training':round(mod_final.score(X_train_tuning, y_train_tuning),3), 
                    'R2_val':round(mod_final.score(X_val_tuning, y_val_tuning),3),
                    'date':date_, 'nbr_obs':df_model.shape[0]})
df_res = pd.DataFrame(li_dico)
print(df_res.shape)
df_res.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                          | 0/205 [00:00<?, ?it/s]<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
  0%|▍                                                                                 | 1/205 [00:00<00:27,  7.43it/s]<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),


2020-10-16 00:00:00
clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2020-10-17 00:00:00


  1%|▊                                                                                 | 2/205 [00:00<00:27,  7.43it/s]<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:13: RuntimeWarning: Mean of empty slice
  empproba_3_Zone_=pd.NamedAgg(column='empproba_3_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:14: RuntimeWarning: Mean of empty slice
  empproba_4_Zone_=pd.NamedAgg(column='empproba_4_Zone', aggfunc=lambda x: np.nanmean(x)),
  1%|█▏                                                                                | 3/205 [00:00<00:27,  7.41it/s]

clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2020-10-18 00:00:00
clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2020-10-19 00:00:00


<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:13: RuntimeWarning: Mean of empty slice
  empproba_3_Zone_=pd.NamedAgg(column='empproba_3_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:14: RuntimeWarning: Mean of empty slice
  empproba_4_Zone_=pd.NamedAgg(column='empproba_4_Zone', aggfunc=lambda x: np.nanmean(x)),
  2%|█▌                                                                                | 4/205 [00:00<00:27,  7.32it/s]<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b9

...

<ipython-input-41-6b94cef929f2>:6: RuntimeWarning: Mean of empty slice
  perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:7: RuntimeWarning: Mean of empty slice
  perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:10: RuntimeWarning: Mean of empty slice
  median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:11: RuntimeWarning: Mean of empty slice
  in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterO

2021-01-14 00:00:00
clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2021-01-15 00:00:00


<ipython-input-41-6b94cef929f2>:6: RuntimeWarning: Mean of empty slice
  perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:7: RuntimeWarning: Mean of empty slice
  perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:10: RuntimeWarning: Mean of empty slice
  median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:11: RuntimeWarning: Mean of empty slice
  in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterO

clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2021-01-16 00:00:00
clIDother    131
Name: ALL_MLPS_99_10cat_, dtype: int64
2021-01-17 00:00:00


<ipython-input-41-6b94cef929f2>:6: RuntimeWarning: Mean of empty slice
  perc_duration_5_Zone_=pd.NamedAgg(column='perc_duration_5_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:7: RuntimeWarning: Mean of empty slice
  perc_duration_4_Zone_=pd.NamedAgg(column='perc_duration_4_Zone', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:8: RuntimeWarning: Mean of empty slice
  activity_50percentile_sec_ = pd.NamedAgg(column='activity_50percentile_sec', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:9: RuntimeWarning: Mean of empty slice
  kurtosis_duration_=pd.NamedAgg(column='kurtosis_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:10: RuntimeWarning: Mean of empty slice
  median_duration_=pd.NamedAgg(column='median_duration', aggfunc=lambda x: np.nanmean(x)),
<ipython-input-41-6b94cef929f2>:11: RuntimeWarning: Mean of empty slice
  in_WG_15mnAfterOpening_whenopen_=pd.NamedAgg(column='in_WG_15mnAfterO

In [ ]:
plt.plot(df_res['date'].tolist(), df_res['R2_val'].tolist());

In [ ]:
plt.plot(df_res['date'].tolist(), df_res['R2_training'].tolist());